In [1]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

In [3]:
train = pd.read_json("input/train.json")

In [3]:
test = pd.read_json("input/test.json")

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [5]:
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=20,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             zoom_range = 0.2,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches

### VGG19

In [24]:
base_model = VGG16(include_top=False,weights='imagenet',classes=1)


In [50]:
base_model.layers#[14].name

In [7]:

def vggmodel(input_shape,lr = 0.0001):
    #First composite layer
    base_model = VGG16(include_top=False,weights='imagenet',input_shape=input_shape,classes=1)
    for layer in base_model.layers[:4]:
        layer.trainable = False
    X = base_model.output
    X = GlobalMaxPooling2D()(X)
    #Flatten layer
    #X = Flatten()(X)
    #First dense layer
    #X = Dense(1024,activation= 'relu')(X)
    #X = BatchNormalization()(X)
    #X = Dropout(0.2)(X)
    #Second dense layer
    X = Dense(1024,activation= 'relu')(X)
    #X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Dense(1024,activation= 'relu')(X)
    #X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    #X = Dense(256,activation= 'relu')(X)
    #X = BatchNormalization()(X)
    #X = Dropout(0.2)(X)
    #Decision layer
    X = Dense(1,activation = 'sigmoid')(X)
    
    model = Model(inputs=base_model.inputs,outputs=X)
    optimizer=Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7):
    #es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]

#callbacks = get_callbacks(filepath=file_path, patience=3)

In [8]:
Vggmodel=vggmodel(input_shape = (75,75,3),lr = 0.00001)
def fitmodel(model,X_train,y_train,X_valid,y_valid,augment = False,epochs = 50,batch_size = 32,filepath = ".model_weights.hdf5"):
    if augment == False:
        result = model.fit(X_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,shuffle = True,
                           validation_data=(X_valid, y_valid),callbacks=get_callbacks(filepath))
    else:
        result = model.fit_generator(data_augmentation(X_train,y_train,batch_size = batch_size),
                                     steps_per_epoch = len(X_train_cv)/batch_size,epochs=epochs,verbose=1,shuffle = True,
                                     validation_data=(X_valid, y_valid),callbacks=get_callbacks(filepath))
    return result

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0         
__________

In [9]:
result_Vgg = fitmodel(Vggmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,epochs = 100,batch_size = 32,filepath = 'model save/Model 3-VGG 19/first.hdf5')

Epoch 1/100
41/40 [==============================] - 33s 808ms/step - loss: 0.6880 - acc: 0.5795 - val_loss: 0.5804 - val_acc: 0.7103
Epoch 2/100
41/40 [==============================] - 26s 629ms/step - loss: 0.5561 - acc: 0.7053 - val_loss: 0.4016 - val_acc: 0.8069
Epoch 3/100
41/40 [==============================] - 25s 622ms/step - loss: 0.4035 - acc: 0.8093 - val_loss: 0.4357 - val_acc: 0.7664
Epoch 4/100
41/40 [==============================] - 26s 628ms/step - loss: 0.3543 - acc: 0.8277 - val_loss: 0.2783 - val_acc: 0.8598
Epoch 5/100
41/40 [==============================] - 26s 623ms/step - loss: 0.3117 - acc: 0.8604 - val_loss: 0.2976 - val_acc: 0.8536
Epoch 6/100
41/40 [==============================] - 26s 622ms/step - loss: 0.3248 - acc: 0.8377 - val_loss: 0.3078 - val_acc: 0.8536
Epoch 7/100
41/40 [==============================] - 26s 632ms/step - loss: 0.3012 - acc: 0.8589 - val_loss: 0.2761 - val_acc: 0.8785
Epoch 8/100
41/40 [==============================] - 26s 633ms

Epoch 62/100
41/40 [==============================] - 26s 625ms/step - loss: 0.1210 - acc: 0.9466 - val_loss: 0.3316 - val_acc: 0.8910
Epoch 63/100
41/40 [==============================] - 26s 624ms/step - loss: 0.1663 - acc: 0.9303 - val_loss: 0.2970 - val_acc: 0.8847
Epoch 64/100
41/40 [==============================] - 25s 622ms/step - loss: 0.1453 - acc: 0.9348 - val_loss: 0.3391 - val_acc: 0.8847
Epoch 65/100
41/40 [==============================] - 26s 624ms/step - loss: 0.1411 - acc: 0.9443 - val_loss: 0.2988 - val_acc: 0.9034
Epoch 66/100
41/40 [==============================] - 26s 622ms/step - loss: 0.1303 - acc: 0.9481 - val_loss: 0.2865 - val_acc: 0.8972
Epoch 67/100
41/40 [==============================] - 26s 624ms/step - loss: 0.1076 - acc: 0.9573 - val_loss: 0.3104 - val_acc: 0.9128
Epoch 68/100
41/40 [==============================] - 26s 624ms/step - loss: 0.1150 - acc: 0.9542 - val_loss: 0.3659 - val_acc: 0.9003
Epoch 69/100
41/40 [==============================] - 2

In [ ]:
vgg = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [10]:
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from sklearn.metrics import log_loss

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	



SyntaxError: invalid character in identifier (<ipython-input-10-0ba0fae71e6a>, line 11)

In [7]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)



In [13]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size)
    genX2 = gen.flow(X1,X2, batch_size=batch_size)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [14]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 32s 1s/step - loss: 0.6953 - acc: 0.5833 - val_loss: 0.4926 - val_acc: 0.7495
Epoch 2/100
24/24 [==============================] - 29s 1s/step - loss: 0.5155 - acc: 0.7449 - val_loss: 0.3744 - val_acc: 0.8336
Epoch 3/100
24/24 [==============================] - 29s 1s/step - loss: 0.3888 - acc: 0.8194 - val_loss: 0.3639 - val_acc: 0.8430
Epoch 4/100
24/24 [==============================] - 29s 1s/step - loss: 0.3237 - acc: 0.8450 - val_loss: 0.2685 - val_acc: 0.8673
Epoch 5/100
24/24 [==============================] - 29s 1s/step - loss: 0.3022 - acc: 0.8698 - val_loss: 0.2496 - val_acc: 0.8860
Epoch 6/100
24/24 [==============================] - 29s 1s/step - loss: 0.2476 - acc: 0.8973 - val_loss: 0.3560 - val_acc: 0.8168
Epoch 7/100
24/24 [==============================] - 29s 1s/step - loss: 0.2658 - acc: 0.8857 - val_loss: 0.2267 - val_acc: 0.9065
Epoch 8/100
24/24 [==============================] - 29s 1s/step - loss: 0.2364 - a

In [15]:
preds

array([ 0.17434557,  0.33020672,  0.01869651, ...,  0.0418326 ,
        0.9970749 ,  0.01072063], dtype=float32)

In [17]:
p = pd.read_csv('sub.csv')
submission = pd.DataFrame()
submission['id']=p['id']
submission['is_iceberg']=preds
submission.to_csv('sub1.csv', index=False)